# Project to AKS

## Reference 

- https://homel.vsb.cz/~vas218/pdf/acs/grammar.pdf
- https://homel.vsb.cz/~vas218/pdf/acs/vasinek-thesis.pdf
- https://homel.vsb.cz/~vas218/acs.html

## Choosen task

- Reduction paradox

## Description

- Try to reduce message and with every step maximaze entropy

## Official description 

- **RePair - maximal anticompression - reduction paradox**
    - Find the smallest possible representation(measured in the number of symbols) of file using the reduction paradox which leads to the largest increase of zero order entropy representation.
    - Heuristics - largest first, random
    - Describe the algorithm and summarize results to a .doc(x) or .pdf report.
    - Prepare a presentation for 10 minutes about your method.
    - Literature: Vasinek, Dissertation thesis (Chapter 5)

## Steps

- Load dataset
- Can pick a subset for faster running (sample it)
- Find every bigram and try to create rule
- Calculate Entropy for new message
- Need to find rule which reduces size of message but increases entropy (harder to compress)
- Find extremes for every file
- Show graphs

In [1]:
import os
import sys

def adding_module_path():
    module_path = os.path.abspath(os.path.sep.join([".."]*1))

    if module_path not in sys.path:
        sys.path.append(module_path)

adding_module_path()

In [2]:
from src.load_data import get_dataset
from src.load_data import DataSets
from src.get_probs import get_sorted_probs_as_df
import numpy as np
import pandas as pd
import time
from src.save import save_both
from enum import Enum
import re
from ast import literal_eval
import random
import plotly.express as px

In [3]:
TEST_NORMALIZATION_SIZE = 10000

In [4]:
def load_dataset(type, normalization=None, random_pick=False):
    data_dna, path_dna = get_dataset(type)
    if normalization is None:
        return data_dna, path_dna
    if random_pick:
        return "".join(np.random.choice(list(data_dna), normalization)), path_dna
    return data_dna[0:normalization], path_dna 

In [5]:
def get_datasets(normalization=None):
    return [
        #(*get_dataset(DataSets.english, normalization), DataSets.english), 
        (*load_dataset(DataSets.dna, normalization), DataSets.dna),
        #(*load_dataset(DataSets.proteins, normalization), DataSets.proteins),
        #(*load_dataset(DataSets.sources, normalization), DataSets.sources),
    ]


In [6]:
test_datasets = get_datasets(TEST_NORMALIZATION_SIZE)

Loading C://Users//proko//Desktop//University//iv//aks//datasets\dna\dna.50MB


In [7]:
test_datasets

[('GATCAATGAGGTGGACACCAGAGGCGGGGACTTGTAAATAACACTGGGCTGTAGGAGTGATGGGGTTCACCTCTAATTCTAAGATGGCTAGATAATGCATCTTTCAGGGTTGTGCTTCTATCTAGAAGGTAGAGCTGTGGTCGTTCAATAAAAGTCCTCAAGAGGTTGGTTAATACGCATGTTTAATAGTACAGTATGGTGACTATAGTCAACAATAATTTATTGTACATTTTTAAATAGCTAGAAGAAAAGCATTGGGAAGTTTCCAACATGAAGAAAAGATAAATGGTCAAGGGAATGGATATCCTAATTACCCTGATTTGATCATTATGCATTATATACATGAATCAAAATATCACACATACCTTCAAACTATGTACAAATATTATATACCAATAAAAAATCATCATCATCATCTCCATCATCACCACCCTCCTCCTCATCACCACCAGCATCACCACCATCATCACCACCACCATCATCACCACCACCACTGCCATCATCATCACCACCACTGTGCCATCATCATCACCACCACTGTCATTATCACCACCACCATCATCACCAACACCACTGCCATCGTCATCACCACCACTGTCATTATCACCACCACCATCACCAACATCACCACCACCATTATCACCACCATCAACACCACCACCCCCATCATCATCATCACTACTACCATCATTACCAGCACCACCACCACTATCACCACCACCACCACAATCACCATCACCACTATCATCAACATCATCACTACCACCATCACCAACACCACCATCATTATCACCACCACCACCATCACCAACATCACCACCATCATCATCACCACCATCACCAAGACCATCATCATCACCATCACCACCAACATCACCACCATCACCAACACCACCATCACCACCACCACCACCATCATCACCACCACCACCATCATCATCACCACCACCGCCATCATCATCGCCACCACCATGACCACCACCATCACAACCATCAC

In [8]:
data_test, data_path, data_type = test_datasets[0]

In [9]:
def filter_func(dic):
    items = list(filter(lambda x: x[1] > 1, dic.items()))
    res = dict(items)
    return res

In [10]:
def find_k_grams_freq(data, max_size_k=2, filter_func=filter_func):
    
    kgrams_dic = {}

    for k in range(2, max_size_k+1):

        for i in range(len(data) - k):

            n_gram = data[i:i+k]
            
            kgrams_dic[n_gram] = kgrams_dic.get(n_gram, 0) + 1
    
    return filter_func(kgrams_dic)

In [11]:
find_k_grams_freq(data_test)

{'GA': 586,
 'AT': 716,
 'TC': 616,
 'CA': 968,
 'AA': 879,
 'TG': 705,
 'AG': 702,
 'GG': 558,
 'GT': 475,
 'AC': 640,
 'CC': 751,
 'GC': 468,
 'CG': 122,
 'CT': 634,
 'TT': 674,
 'TA': 504}

In [12]:
import math
from collections import Counter

def calc_freq(content):
    c = Counter(list(content))
    return c

def calc_p(counter, n):
    counter = dict(counter)
    res = {}
    for k, v in counter.items():
        res[k] = v / n  
    return res

def get_n(counter):
    counter = dict(counter)
    return np.sum(list(counter.values()))

def calc_H(p):
    H = 0
    for k, v in p.items():
        #Shannon equation!
        H += p[k] * math.log2(p[k])
    return -H

def calc_entropy_for_message(message):
    counter = calc_freq(message)
    n = get_n(counter)
    p = calc_p(counter, n)
    H = calc_H(p)
    return H

In [13]:
calc_entropy_for_message(data_test)

1.9895772725667296

In [14]:
def diff_entropy(message1, message2):
    message1_entropy = calc_entropy_for_message(message1)
    message2_entropy = calc_entropy_for_message(message2)
    message_1_entropy_size = message1_entropy * len(message1)
    message_2_entropy_size = message2_entropy * len(message2)


    diff = message_1_entropy_size - message_2_entropy_size
    #print(f'{round(message1_entropy, 2)} * {len(message1)} < {round(message2_entropy, 2)} * {len(message2)} ... {diff}')
    return diff

In [15]:
diff_entropy(data_test, data_test)

0.0

In [16]:
all_chars_uni = tuple(chr(i) for i in range(32, 0x110000) if chr(i).isprintable())

In [17]:
all_chars_ascii = list(range(0, 256))
all_chars_ascii = [chr(ascii_char) for ascii_char in all_chars_ascii]

In [18]:
def find_not_existing_character(current_alpahbet, gen_chars=all_chars_uni):
    can_use = set(gen_chars).difference(set(current_alpahbet))
    ascii_picked_char = random.choice(list(can_use))
    return ascii_picked_char

In [19]:
find_not_existing_character(['A', 'C', 'G', 'T'])

'𫜊'

In [20]:
def transform_message(message, ngram_for_replace):
    current_alphabet_size = np.unique(list(message))

    #print('Alphabet size', len(current_alphabet_size))

    replace_character = find_not_existing_character(current_alphabet_size)
    
    return message.replace(ngram_for_replace, replace_character), replace_character

In [21]:
def calculate_for_ngrams_diff(ngrams, message, method_entropy=diff_entropy, init_message=None):
    res = {}
    
    for k, v in ngrams.items():
        res[k] = {
            "Counter": v,
            #message - current
            #message - next message
            #"Diff": method_entropy(init_message, transform_message(message, k)[0])
            "Diff": method_entropy(message, transform_message(message, k)[0])
        }
    return res

In [22]:
r = calculate_for_ngrams_diff(
        find_k_grams_freq(data_test),
        data_test,
        diff_entropy,
        data_test
    )

pd.DataFrame.from_dict(
    r, 
    orient="index"
)

,Counter,Diff
GA,586,-686.909129
AT,716,-794.272380
TC,616,-679.714642
CA,968,-488.571041
AA,879,-753.006409
TG,705,-393.334248
AG,702,-586.882909
GG,558,-314.800226
GT,475,-611.867113
AC,640,-831.446036


In [23]:
r

{'GA': {'Counter': 586, 'Diff': -686.9091292880221},
 'AT': {'Counter': 716, 'Diff': -794.2723799380256},
 'TC': {'Counter': 616, 'Diff': -679.7146417421354},
 'CA': {'Counter': 968, 'Diff': -488.5710412647459},
 'AA': {'Counter': 879, 'Diff': -753.0064090554151},
 'TG': {'Counter': 705, 'Diff': -393.33424757568355},
 'AG': {'Counter': 702, 'Diff': -586.882908578893},
 'GG': {'Counter': 558, 'Diff': -314.80022619213923},
 'GT': {'Counter': 475, 'Diff': -611.8671131922092},
 'AC': {'Counter': 640, 'Diff': -831.4460359297373},
 'CC': {'Counter': 751, 'Diff': -415.72576803416814},
 'GC': {'Counter': 468, 'Diff': -607.6780573412434},
 'CG': {'Counter': 122, 'Diff': -421.589100361467},
 'CT': {'Counter': 634, 'Diff': -666.1015642405691},
 'TT': {'Counter': 674, 'Diff': -552.6838623637159},
 'TA': {'Counter': 504, 'Diff': -873.9444092367921}}

In [24]:
r.items()

dict_items([('GA', {'Counter': 586, 'Diff': -686.9091292880221}), ('AT', {'Counter': 716, 'Diff': -794.2723799380256}), ('TC', {'Counter': 616, 'Diff': -679.7146417421354}), ('CA', {'Counter': 968, 'Diff': -488.5710412647459}), ('AA', {'Counter': 879, 'Diff': -753.0064090554151}), ('TG', {'Counter': 705, 'Diff': -393.33424757568355}), ('AG', {'Counter': 702, 'Diff': -586.882908578893}), ('GG', {'Counter': 558, 'Diff': -314.80022619213923}), ('GT', {'Counter': 475, 'Diff': -611.8671131922092}), ('AC', {'Counter': 640, 'Diff': -831.4460359297373}), ('CC', {'Counter': 751, 'Diff': -415.72576803416814}), ('GC', {'Counter': 468, 'Diff': -607.6780573412434}), ('CG', {'Counter': 122, 'Diff': -421.589100361467}), ('CT', {'Counter': 634, 'Diff': -666.1015642405691}), ('TT', {'Counter': 674, 'Diff': -552.6838623637159}), ('TA', {'Counter': 504, 'Diff': -873.9444092367921})])

In [25]:
def pick_largest(items):
    return list(sorted(items, key=lambda x: x[1]['Diff']))[0]

def pick_random(items):
    return random.choice(items)

def pick_only_decreasing(dic, pick_method=pick_largest):
    items = dic.items()

    decreasing_items = list(filter(lambda x: x[1]['Diff'] < 0, items))

    if len(decreasing_items) == 0:
        return None

    return pick_method(decreasing_items)    

In [26]:
pick_only_decreasing(r, pick_largest)

('TA', {'Counter': 504, 'Diff': -873.9444092367921})

# Algorithm implementation - Reduction paradox

In [27]:
class TableFields(Enum):
    Rule = "Rule"
    EntropyMove = "EntropyMove"
    CurrentEntropy = "CurrentEntropy"
    EntropySize = "EntropySize"
    DataType = "DataType"
    DescriptionData = "DescriptionData"
    MessageSize = "MessageSize"
    AlphabetSize = "AlphabetSize"
    GrammaticSize = "GrammaticSize"
    CalcTime = "CalcTime"


In [28]:
def create_value(message_0, message_1, diff, replace_character, n_gram, grammatic, tic, type_data=None, description_data=None):
    new_message_alphabet_size = len(np.unique(list(message_1)))
    new_message_size = len(message_1)
    new_message_entropy = calc_entropy_for_message(message_1)

    new_rule = f"{n_gram} -> {replace_character}"
    grammatic[n_gram] = replace_character

    tac = time.time()
    
    return {
        TableFields.Rule.value: new_rule,
        TableFields.EntropyMove.value: diff,
        TableFields.CurrentEntropy.value: new_message_entropy,
        TableFields.EntropySize.value: new_message_entropy * new_message_size,
        TableFields.DataType.value: type_data,
        TableFields.DescriptionData.value: description_data,
        TableFields.MessageSize.value: new_message_size,
        TableFields.AlphabetSize.value: new_message_alphabet_size,
        TableFields.GrammaticSize.value: len(list(grammatic.keys())),
        TableFields.CalcTime.value: tac - tic,
    }


In [29]:
def algorithm_step(init_message, message, grammatic, type_data=None, description_data=None, tic=None, heuristics_method=pick_largest, optimized=False):
    #Find ngrams
    n_grams = find_k_grams_freq(message)
    
    #optimized = diff_entropy change pls use formula

    diff_table = calculate_for_ngrams_diff(
        n_grams,
        message,
        diff_entropy,
        init_message
    )

    picked = pick_only_decreasing(diff_table, heuristics_method)
    print(picked)

    if picked is None:
        return None

    #('AT', {'Counter': 867, 'Diff': -0.2814694848677286})
    n_gram, dic_values = picked
    transformed_message, replace_character = transform_message(message, n_gram)
    
    return transformed_message, create_value(message, transformed_message, dic_values['Diff'], replace_character, n_gram, grammatic, tic, type_data, description_data)


In [30]:
def algorithm(message, type_data=None, description_data=None, limit_step=None, heuristic_method=pick_largest, optimized=False):
    init_message = message

    res = {}
    grammatic = {}
    step = 0
#message_0, message_1, diff, replace_character, n_gram, grammatic, tic, type_data=None, description_data=None
    step_value = create_value(init_message, init_message, 0, "", "", grammatic, time.time(), type_data)

    res[step] = step_value
    step += 1

    while True:
        tic = time.time()
        if limit_step is not None and limit_step == step:
            break
        step_value = algorithm_step(init_message, message, grammatic, type_data, description_data, tic, heuristic_method, optimized)
        print(step, len(message))
        print('\n')
        if step_value is None:
            break

        else:
            transformed_message, value = step_value
            message = transformed_message

            step += 1
            res[step] = value

    return res

## Testing data

In [31]:
test_result = algorithm(data_test, limit_step=20)

('TA', {'Counter': 504, 'Diff': -873.9444092367921})
1 10000


('AC', {'Counter': 544, 'Diff': -739.9214842923175})
2 9496


('GT', {'Counter': 386, 'Diff': -560.4334175307667})
3 8952


('CA', {'Counter': 542, 'Diff': -372.54555005118164})
4 8566


('AT', {'Counter': 274, 'Diff': -317.46880517862155})
5 8024


('GC', {'Counter': 344, 'Diff': -326.1923034041829})
6 7750


('TG', {'Counter': 280, 'Diff': -266.60311892627215})
7 7405


('CG', {'Counter': 38, 'Diff': -131.08959485316518})
8 7125


('𗝸T', {'Counter': 66, 'Diff': -106.07417463021557})
9 7087


('G𐋥', {'Counter': 59, 'Diff': -97.77619238536136})
10 7021


('𭛿A', {'Counter': 106, 'Diff': -93.8192535860253})
11 6962


('A𗝸', {'Counter': 65, 'Diff': -87.73675945264404})
12 6856


('𐋥C', {'Counter': 54, 'Diff': -86.7758646814109})
13 6791


('CT', {'Counter': 231, 'Diff': -85.43774104089971})
14 6737


('G𭛿', {'Counter': 81, 'Diff': -76.51381828860758})
15 6506


('𗝸A', {'Counter': 64, 'Diff': -72.762703708755})
16 6425


('𐋥G',

In [32]:
df = pd.DataFrame.from_dict(test_result, orient="index")
df

,Rule,EntropyMove,CurrentEntropy,EntropySize,DataType,DescriptionData,MessageSize,AlphabetSize,GrammaticSize,CalcTime
0,->,0.000000,1.989577,19895.772726,None,None,10000,4,1,0.007998
2,TA -> 𐋥,-873.944409,2.187207,20769.717135,None,None,9496,5,2,0.445476
3,AC -> 𗝸,-739.921484,2.402775,21509.638619,None,None,8952,6,3,0.607649
4,GT -> 𘥗,-560.433418,2.576474,22070.072037,None,None,8566,7,4,0.891332
5,CA -> 𭛿,-372.545550,2.796936,22442.617587,None,None,8024,8,5,1.008617
6,AT -> 𐫅,-317.468805,2.936785,22760.086392,None,None,7750,9,6,1.427192
7,GC -> 𐚣,-326.192303,3.117661,23086.278695,None,None,7405,10,7,1.715136
8,TG -> 𐠞,-266.603119,3.277597,23352.881814,None,None,7125,11,8,1.948813
9,CG -> 𒈡,-131.089595,3.313669,23483.971409,None,None,7087,12,9,2.283151
10,𗝸T -> 疢,-106.074175,3.359927,23590.045584,None,None,7021,13,10,2.516385


In [33]:
fig = px.line(df, x=df.index, y=df.EntropyMove, text=[("%.1f" % x) for x in df.EntropyMove.values], title='Pohyb entropie')
fig.show()

In [34]:
fig = px.line(df, x=df.index, y=df.MessageSize, text=df.MessageSize, title='Velikost zprávy')
fig.show()
fig.write_image("test.png")

In [35]:
fig = px.line(df, x=df.index, y=[df.MessageSize, df.EntropySize], title='Velikost zprávy proti aktuální entropii')
fig.show()

In [36]:
fig = px.line(df, x=df.index, y=df.EntropySize, title='H*len(m)', text=[("%.0f" % x) for x in df.EntropySize.values])
fig.update_traces(textposition='top center')
fig.show()

# Real experiment data

In [46]:
def get_datasets(normalization=None):
    return [
        #(*load_dataset(DataSets.english, normalization), DataSets.english),  done
        #(*load_dataset(DataSets.dna, normalization), DataSets.dna),
        #(*load_dataset(DataSets.proteins, normalization), DataSets.proteins),
        (*load_dataset(DataSets.sources, normalization), DataSets.sources),
    ]

In [47]:
NORM_VALUES = [10000]
CSV_NAME = "steps.csv"
ENTROPY_GRAPH = "entropy_paradox.png"
MESSAGE_GRAPH = "message_size.png"
MESSAGE_SIZE_ENTROPY_GRAPH = "message_entropy_size.png"
MESSAGE_ENTROPY = "message_entropy.png"

In [48]:
def write_images(df, path):
    fig = px.line(df, x=df.index, y=df.EntropyMove, text=[("%.1f" % x) for x in df.EntropyMove.values], title='Pohyb entropie')
    fig.update_traces(textposition='top center')
    fig.write_image(os.path.sep.join([path, ENTROPY_GRAPH]))
    
    fig = px.line(df, x=df.index, y=df.MessageSize, text=df.MessageSize, title='Velikost zprávy')
    fig.update_traces(textposition='top center')
    fig.write_image(os.path.sep.join([path, MESSAGE_GRAPH]))    

    fig = px.line(df, x=df.index, y=[df.MessageSize, df.EntropySize], title='Velikost zprávy proti aktuální entropii')
    fig.update_traces(textposition='top center')
    fig.write_image(os.path.sep.join([path, MESSAGE_SIZE_ENTROPY_GRAPH]))

    fig = px.line(df, x=df.index, y=df.EntropySize, title='H*len(m)', text=[("%.0f" % x) for x in df.EntropySize.values])
    fig.update_traces(textposition='top center')   
    fig.write_image(os.path.sep.join([path, MESSAGE_ENTROPY]))

In [49]:
def save_dataframe(df, path):
    path = os.path.sep.join([path, CSV_NAME])
    df.to_csv(path, index=False)

In [50]:
def run_algorithm_for_datasets(normalization_values=NORM_VALUES, largest=True, limit_steps=None, optimized=False):
    for n_v in normalization_values:

        datasets = get_datasets(n_v)

        for data, data_path, data_type in datasets:
            data_type_string = data_type.value
            current_path = os.path.sep.join([data_type_string, str(n_v)])


            limit_steps_str = "None" if limit_steps is None else str(limit_steps)
            optimized_string = "optimized" if optimized else "bruto"

            steps_path = os.path.sep.join([current_path, limit_steps_str, str(largest), optimized_string])

            if not os.path.isdir(steps_path):
                os.makedirs(steps_path)

            heurestic_method = pick_largest if largest else pick_random

            res = algorithm(data, data_type_string, "", limit_step=limit_steps, heuristic_method=heurestic_method, optimized=optimized)
            df = pd.DataFrame.from_dict(res, orient="index")




            save_dataframe(df, steps_path)
            write_images(df, steps_path)

In [51]:
LIMIT_STEPS = [None]
LARGEST_METHOD = [True] #largest || random
OPTIMIZED = [False] #not optimized (bruto) || optimized

In [52]:
import itertools

In [53]:
exps = list(itertools.product(LIMIT_STEPS, LARGEST_METHOD, OPTIMIZED))
exps

[(None, True, False)]

In [54]:
for limit_steps, largest_method, optimized in exps:
    run_algorithm_for_datasets(NORM_VALUES, largest_method, limit_steps, optimized)

Loading C://Users//proko//Desktop//University//iv//aks//datasets\sources\sources.50MB
('  ', {'Counter': 113, 'Diff': -142.9876062815747})
1 10000


('r ', {'Counter': 36, 'Diff': -62.80239439949219})
2 9934


(' t', {'Counter': 38, 'Diff': -58.552960385182814})
3 9898


('e ', {'Counter': 73, 'Diff': -57.76897615909547})
4 9860


('n ', {'Counter': 29, 'Diff': -45.28772176144412})
5 9787


('\n ', {'Counter': 45, 'Diff': -38.43491557033849})
6 9758


(' e', {'Counter': 11, 'Diff': -35.34708525164751})
7 9713


('it', {'Counter': 21, 'Diff': -34.93004967402521})
8 9702


(' d', {'Counter': 21, 'Diff': -32.6101054637038})
9 9681


(' n', {'Counter': 13, 'Diff': -32.108864506975806})
10 9660


('ri', {'Counter': 36, 'Diff': -31.671712671901332})
11 9647


('i ', {'Counter': 8, 'Diff': -29.890893589348707})
12 9611


(' a', {'Counter': 14, 'Diff': -28.066010088507028})
13 9603


('te', {'Counter': 16, 'Diff': -27.18590813089395})
14 9589


(' r', {'Counter': 8, 'Diff': -26.28743290069542}